## Generate countries dataset

In [2]:
import pandas as pd
import pickle

In [4]:
cities_ds = "../../datasets/cities_questions_augmented/all_cities.csv"
cities = pd.read_csv(cities_ds, sep=';')
cities = cities[['ASCII Name', 'Country name EN', 'Population']]
cities.columns = ['city', 'country', 'population']
cities = cities[cities['population'] != 0]
cities = cities.drop_duplicates(subset=['city'], keep=False)
#cities = cities[isinstance(cities['city'], str)]
cities = cities[cities.apply(lambda row : isinstance(row['city'], str), axis=1)]
cities = cities[cities.apply(lambda row : isinstance(row['country'], str), axis=1)]
print(len(cities))
cities.head()
#df['is_correct'] = df.apply(lambda row : row['filtered_answer'] in country_names[row['correct_country']], axis=1) 

110826


,city,country,population
0,Huayllati,Peru,515
1,Duraznopampa,Peru,249
2,Kimbiri,Peru,4369
3,Urb. Santo Domingo,Peru,5000
6,Kimbe,Papua New Guinea,18847


In [ ]:
selection = cities.sample(10000)

In [ ]:
cities_questions_augmented = pd.DataFrame(columns=["statement", "label", "city", "country", "correct_country", "question", "question_with_answer"])
for index, row in selection.iterrows():
    row = {"statement": "The city of " + row['city'] + " is in " + row['country'] + ".", 
            "label": 1, 
            "city": row['city'], 
            "country": row['country'], 
            "correct_country": row['country'], 
            "question": "In which country is the city of " + row['city'] + " located?", 
            "question_with_answer": "In which country is the city of " + row['city'] + " located? " + row['country'] + "."}
    row_df = pd.Series(row).to_frame().T
    cities_questions_augmented = pd.concat([cities_questions_augmented, row_df], axis=0, ignore_index=True)

In [ ]:
cities_questions_augmented

In [ ]:
cities_questions_augmented.to_csv("../../datasets/cities_expansions/cities_questions_augmented_2.csv")

## Create countries dictionary

In [3]:
set(cities['country'])
country_names = {}
for country in set(cities['country']):
    country_names[country] = [country]
country_names['Russian Federation'].append('Russia')
country_names['Viet Nam'].append('Vietnam')
country_names['Venezuela, Bolivarian Rep. of'].append('Venezuela')
country_names['United States'].append('USA')
country_names['United States'].append('the USA')
country_names['United States'].append('The USA')
country_names['United States'].append('The United States')
country_names['Congo, Democratic Republic of the'].append('Congo')
country_names['Hong Kong, China'].append('Hong Kong')
country_names['Iran, Islamic Rep. of'].append('Iran')
country_names['West Bank and Gaza Strip'].append('Palestine')
country_names['United Kingdom'].append('UK')
country_names['United Kingdom'].append('the UK')
country_names['United Kingdom'].append('The UK')
country_names["Korea, Dem. People's Rep. of"].append('North Korea')
country_names['Korea, Republic of'].append('South Korea')
country_names['Sudan, The Republic of'].append('Sudan')

In [4]:
with open('../../datasets/cities_expansions/countries_dictionary.pkl', 'wb') as f:
    pickle.dump(country_names, f)

## Create 'statement' dataset

In [19]:
cities = pd.read_csv("../../datasets/cities_expansions/cities_questions_augmented_2.csv")

In [20]:
cities.head()

,Unnamed: 0,statement,label,city,country,correct_country,question,question_with_answer
0,0,The city of Kostomloty Pierwsze is in Poland.,1,Kostomloty Pierwsze,Poland,Poland,In which country is the city of Kostomloty Pie...,In which country is the city of Kostomloty Pie...
1,1,The city of Bognankro is in Côte d'Ivoire.,1,Bognankro,Côte d'Ivoire,Côte d'Ivoire,In which country is the city of Bognankro loca...,In which country is the city of Bognankro loca...
2,2,The city of Le Raincy is in France.,1,Le Raincy,France,France,In which country is the city of Le Raincy loca...,In which country is the city of Le Raincy loca...
3,3,The city of Tobol is in Kazakhstan.,1,Tobol,Kazakhstan,Kazakhstan,In which country is the city of Tobol located?,In which country is the city of Tobol located?...
4,4,The city of Tayabas Ibaba is in Philippines.,1,Tayabas Ibaba,Philippines,Philippines,In which country is the city of Tayabas Ibaba ...,In which country is the city of Tayabas Ibaba ...


In [21]:
len(cities)

9984

In [22]:
cities_new = cities[['question']]

In [23]:
cities_new.columns = ['statement']

In [24]:
cities_new.to_csv("../../datasets/cities_questions_augmented_2_for_activations.csv")

Then run: 
python3 generate_acts.py --model_family Llama3 --model_size 8B --model_type chat --layers 12 --datasets cities_questions_augmented_2_for_activations --device cuda:0 --output_dir acts/
